In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

## Prophet Feature Selection, so Dropping All Lag and Rolling

In [2]:
covid_df = pd.read_csv('illinois_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})
covid_df = covid_df.drop(['new_cases_1 lag', 'new_cases_2_lag', 'new_cases_3_lag', 'new_cases_4_lag', 'new_cases_7_lag', 'new_cases_30_lag', 'new_cases_50_lag', 'new_cases_150_lag', 'new_cases_350_lag', 'new_deaths_1_lag', 'new_deaths_2_lag', 'new_deaths_3_lag', 'new_deaths_4_lag', 'new_deaths_7_lag', 'new_deaths_30_lag', 'new_deaths_50_lag'], axis=1)

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size] # notice here the original test test is never used

In [3]:
train_size

671

In [4]:
X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
y_train = covid_train['y']

In [5]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_1 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_1[1].to_list())

['PEOPLE_DEATH_NEW_COUNT', 'new_cases_50_max', 'new_cases_10_mean', 'new_cases_10_max', 'new_cases_10_std', 'new_hospitalized_patients', 'new_deaths_10_mean', 'new_cases_50_std', 'Series_Complete_65Plus', 'new_cases_100_max', 'Series_Complete_Yes', 'Series_Complete_5Plus', 'relative_humidity', 'Booster_Doses', 'average_temperature_celsius', 'minimum_temperature_celsius', 'Administered_Dose1_Recip', 'vaccination_rate_complete', 'new_cases_100_std', 'rainfall_mm', 'Series_Complete_18Plus', 'Administered_Dose1_Recip_5Plus', 'maximum_temperature_celsius', 'new_deaths_10_std', 'dew_point', 'cancel_public_events', 'Administered_Dose1_Recip_12Plus', 'new_cases_50_min', 'Booster_Doses_65Plus', 'new_deaths_50_std']


In [6]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_2 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_2[1].to_list())

['new_cases_10_std', 'new_cases_10_mean', 'PEOPLE_DEATH_NEW_COUNT', 'new_hospitalized_patients', 'new_cases_10_max', 'new_deaths_10_std', 'relative_humidity', 'new_cases_50_std', 'new_cases_100_std', 'Administered_Dose1_Recip_65Plus', 'rainfall_mm', 'maximum_temperature_celsius', 'dew_point', 'new_cases_50_mean', 'new_deaths_10_mean', 'minimum_temperature_celsius', 'average_temperature_celsius', 'current_intensive_care_patients', 'new_deaths_50_std', 'Administered_Dose1_Recip', 'Administered_Dose1_Recip_5Plus', 'new_cases_10_min', 'Booster_Doses', 'new_deaths_100_std', 'new_deaths_100_mean', 'new_deaths_50_mean', 'new_cases_50_max', 'new_cases_100_mean', 'Booster_Doses_65Plus', 'new_deaths_10_max']


In [7]:
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier

model_c = GradientBoostingRegressor().fit(X_train, y_train)
predictors_3 = pd.concat([pd.Series(model_c.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_3[1].to_list())

['new_cases_10_mean', 'new_cases_10_max', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_std', 'new_cases_50_max', 'Series_Complete_65Plus', 'new_deaths_10_mean', 'new_hospitalized_patients', 'Booster_Doses_18Plus', 'new_deaths_100_std', 'Booster_Doses_65Plus', 'relative_humidity', 'Series_Complete_18Plus', 'new_cases_10_min', 'new_deaths_10_std', 'new_deaths_50_std', 'average_temperature_celsius', 'maximum_temperature_celsius', 'Administered_Dose1_Recip_65Plus', 'current_intensive_care_patients', 'dew_point', 'school_closing', 'new_deaths_100_mean', 'new_cases_100_mean', 'Series_Complete_12Plus', 'new_deaths_50_mean', 'rainfall_mm', 'Series_Complete_5Plus', 'new_cases_100_max', 'new_cases_100_min']


In [8]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier

model_d = AdaBoostRegressor(random_state=1, base_estimator = DecisionTreeRegressor(random_state=1)).fit(X_train, y_train)
predictors_4 = pd.concat([pd.Series(model_d.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_4[1].to_list())

['new_cases_10_std', 'PEOPLE_DEATH_NEW_COUNT', 'Series_Complete_65Plus', 'new_cases_10_max', 'new_cases_10_mean', 'Administered_Dose1_Recip_12Plus', 'new_cases_50_max', 'new_cases_50_std', 'minimum_temperature_celsius', 'Series_Complete_Yes', 'new_deaths_10_std', 'new_hospitalized_patients', 'vaccination_rate_complete', 'maximum_temperature_celsius', 'relative_humidity', 'new_deaths_10_mean', 'dew_point', 'Booster_Doses_18Plus', 'booster_rate', 'Administered_Dose1_Recip', 'average_temperature_celsius', 'new_cases_100_mean', 'new_deaths_50_std', 'rainfall_mm', 'Administered_Dose1_Recip_65Plus', 'new_deaths_50_mean', 'new_deaths_100_mean', 'current_intensive_care_patients', 'Series_Complete_18Plus', 'new_deaths_100_std']


In [9]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier

model_e = RandomForestRegressor().fit(X_train, y_train)
predictors_5 = pd.concat([pd.Series(model_e.feature_importances_), pd.Series(X_train.columns)], axis=1).sort_values(by=0, ascending = False).head(30)
print(predictors_5[1].to_list())

['new_cases_10_mean', 'new_cases_10_std', 'new_cases_50_max', 'PEOPLE_DEATH_NEW_COUNT', 'Series_Complete_65Plus', 'new_cases_10_max', 'new_hospitalized_patients', 'new_cases_50_std', 'Series_Complete_5Plus', 'minimum_temperature_celsius', 'Series_Complete_18Plus', 'new_deaths_10_mean', 'new_deaths_100_mean', 'dew_point', 'relative_humidity', 'new_deaths_10_std', 'vaccination_rate_complete', 'average_temperature_celsius', 'current_intensive_care_patients', 'new_cases_100_mean', 'vaccination_rate_1dose', 'Administered_Dose1_Recip_5Plus', 'Administered_Dose1_Recip', 'new_cases_50_mean', 'Series_Complete_Yes', 'Booster_Doses_18Plus', 'Administered_Dose1_Recip_12Plus', 'new_cases_10_min', 'Booster_Doses_65Plus', 'new_cases_100_std']


In [10]:
# using MARS summary for feature selection
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss') # note, terms in brackets are the hyperparameters 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
--------------------------------------------------------------------
Basis Function                                 Pruned  Coefficient  
--------------------------------------------------------------------
(Intercept)                                    No      39028.4      
h(new_cases_10_mean-10676.5)                   No      2.9151       
h(10676.5-new_cases_10_mean)                   No      -1.25135     
h(new_cases_10_std-7026.32)                    Yes     None         
h(7026.32-new_cases_10_std)                    No      -0.328222    
h(booster_rate-0.208955)                       Yes     None         
h(0.208955-booster_rate)                       Yes     None         
h(new_cases_10_mean-11699.9)                   Yes     None         
h(11699.9-new_cases_10_mean)                   Yes     None         
PEOPLE_DEATH_NEW_COUNT                         Yes     None         
h(new_deaths_10_max-258)                       Yes     None         
h(258-new_deaths_10_ma

In [11]:
def common_member(a, b, c, d, e):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
    set_4 = set(d)
    set_5 = set(e)
 
    if (set_1 & set_2 & set_3 & set_4 & set_5):
        print(set_1 & set_2 & set_3 & set_4 & set_5)


In [12]:
a = predictors_1[1].to_list()
b = predictors_2[1].to_list()
c = predictors_3[1].to_list()
d = predictors_4[1].to_list()
e = predictors_5[1].to_list()

In [13]:
full = common_member(a, b, c, d, e)
full

{'new_hospitalized_patients', 'new_cases_50_max', 'new_deaths_10_mean', 'average_temperature_celsius', 'new_cases_10_std', 'new_cases_10_mean', 'new_cases_10_max', 'new_deaths_10_std', 'relative_humidity', 'PEOPLE_DEATH_NEW_COUNT', 'dew_point'}


## XGBoost Feature Selection, includes lag and rolling

In [ ]:
import os
os._exit(00)

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
covid_df = pd.read_csv('illinois_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size]

X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
X_train = X_train.astype('float32')
y_train = covid_train['y']

In [3]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_7 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(40)
print(predictors_7[1].to_list())

['new_cases_7_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_50_max', 'new_hospitalized_patients', 'new_cases_10_mean', 'new_cases_10_max', 'new_cases_1 lag', 'new_cases_50_std', 'new_cases_10_std', 'new_deaths_1_lag', 'new_cases_2_lag', 'Booster_Doses', 'new_cases_150_lag', 'new_deaths_7_lag', 'new_cases_100_max', 'Booster_Doses_18Plus', 'stringency_index', 'new_cases_50_lag', 'vaccination_rate_complete', 'new_deaths_4_lag', 'new_cases_50_mean', 'relative_humidity', 'new_cases_4_lag', 'dew_point', 'new_deaths_50_std', 'Administered_Dose1_Recip_18Plus', 'Series_Complete_65Plus', 'maximum_temperature_celsius', 'new_cases_3_lag', 'Administered_Dose1_Recip_12Plus', 'new_deaths_2_lag', 'Series_Complete_5Plus', 'Series_Complete_12Plus', 'new_deaths_10_std', 'new_deaths_100_mean', 'cancel_public_events', 'Booster_Doses_50Plus', 'new_cases_100_std', 'debt_relief', 'booster_rate']


In [4]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_8 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_8[1].to_list())

['new_cases_7_lag', 'new_cases_10_std', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_mean', 'new_cases_10_max', 'new_cases_1 lag', 'relative_humidity', 'new_cases_3_lag', 'new_cases_2_lag', 'new_hospitalized_patients', 'new_cases_30_lag', 'new_cases_4_lag', 'new_deaths_1_lag', 'new_deaths_4_lag', 'new_deaths_2_lag', 'new_deaths_10_std', 'new_cases_50_std', 'new_deaths_7_lag', 'rainfall_mm', 'maximum_temperature_celsius', 'new_deaths_50_lag', 'new_deaths_30_lag', 'new_cases_150_lag', 'new_cases_50_lag', 'new_deaths_3_lag', 'current_intensive_care_patients', 'new_cases_350_lag', 'minimum_temperature_celsius', 'Administered_Dose1_Recip_65Plus', 'Administered_Dose1_Recip']


In [5]:
model_g = xgb.XGBRegressor().fit(X_train, y_train, verbose=False)
predictors_6 = pd.concat([pd.Series(model_g.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_6[1].to_list())

['new_cases_7_lag', 'new_cases_10_max', 'new_cases_10_std', 'new_cases_50_max', 'new_cases_10_mean', 'Booster_Doses', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_1 lag', 'Booster_Doses_65Plus', 'new_cases_150_lag', 'new_deaths_100_max', 'new_deaths_4_lag', 'Series_Complete_Yes', 'new_deaths_50_min', 'new_cases_50_lag', 'new_cases_3_lag', 'Completeness_pct', 'stringency_index', 'new_hospitalized_patients', 'Booster_Doses_50Plus', 'new_cases_4_lag', 'new_deaths_50_lag', 'new_cases_100_mean', 'Administered_Dose1_Recip_5Plus', 'new_deaths_30_lag', 'new_cases_50_mean', 'new_cases_100_min', 'new_deaths_7_lag', 'Administered_Dose1_Recip', 'new_deaths_10_max']


In [6]:
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss', allow_missing=True) 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
-------------------------------------------------------------------------
Basis Function                                      Pruned  Coefficient  
-------------------------------------------------------------------------
(Intercept)                                         No      18204.5      
present(new_cases_7_lag)                            No      284007       
missing(new_cases_7_lag)                            Yes     None         
h(new_cases_7_lag-12384)*present(new_cases_7_lag)   Yes     None         
h(12384-new_cases_7_lag)*present(new_cases_7_lag)   No      2.22538      
new_cases_10_max                                    Yes     None         
h(PEOPLE_DEATH_NEW_COUNT-189)                       No      18.9245      
h(189-PEOPLE_DEATH_NEW_COUNT)                       No      -20.7925     
h(new_deaths_10_max-258)                            No      194.15       
h(258-new_deaths_10_max)                            Yes     None         
present(new_cases_4_lag)  

In [7]:
def common_member(a, b, c):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
 
    if (set_1 & set_2 & set_3):
        print(set_1 & set_2 & set_3)


In [8]:
a = predictors_7[1].to_list()
b = predictors_8[1].to_list()
c = predictors_6[1].to_list()

In [9]:
full = common_member(a, b, c)
full

{'PEOPLE_DEATH_NEW_COUNT', 'new_cases_1 lag', 'new_hospitalized_patients', 'new_cases_10_mean', 'new_cases_10_std', 'new_cases_10_max', 'new_cases_4_lag', 'new_cases_150_lag', 'new_cases_50_lag', 'new_deaths_4_lag', 'new_cases_7_lag', 'new_deaths_7_lag', 'new_cases_3_lag'}
